In [1]:
import numpy as np
import pandas as pd

import os

In [3]:
import pandas as pd

pd.set_option('display.max_rows', 100)

df_train = pd.read_csv("data_train.csv")

df_test = pd.read_csv("data_test.csv")

sub = pd.read_csv("sample_submission.csv")

%matplotlib inline
pd.set_option('display.max_columns', None)

In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_validate

import matplotlib.pyplot as plt
from itertools import cycle

from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix, accuracy_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

import numpy as np
from tqdm import tqdm

In [5]:
from sklearn import preprocessing

#le = preprocessing.LabelEncoder()
categ = df_train.select_dtypes(include = "object").columns

for feat in categ:
    df_train[feat] = df_train[feat].astype("category") #le.fit_transform(df_train[feat].astype(str))

In [6]:
df_train.drop(columns = df_train.select_dtypes(include = "float").loc[:, df_train.select_dtypes(include = "float").std() < .1], inplace=True)

In [7]:
X = df_train[df_train.columns[df_train.columns!="class"]]
y = df_train[df_train.columns[df_train.columns=="class"]]

In [11]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder 

lh = LabelEncoder()
y_lh = lh.fit_transform(y)

C:\Users\ffede\Venv\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [ ]:
n_samples, n_features = X.shape
n_splits = 5
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

classifier = LGBMClassifier(learning_rate=0.1, n_estimators=300,
                        min_child_weight=3, subsample=0.6, colsample_bytree=1.0, 
                        seed=1)

#classifier = LGBMClassifier()

mean_tpr = 0.0
mean_fpr = np.linspace(0, 1, 100)

acc = 0
for (train, test) in tqdm(cv.split(X, y), total = n_splits):
    cl = classifier.fit(X.iloc[train], y.iloc[train])
    probas_ = cl.predict_proba(X.iloc[test])
    local = accuracy_score(y_lh[test],np.argmax(probas_, axis=1))
    print(local)
    acc += local
    
acc = acc / n_splits
print("Mean accuracy", acc)

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]C:\Users\ffede\Venv\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [42]:
classifier = LGBMClassifier()
classifier.fit(X, y)

In [44]:
for feat in categ[1:]:
    df_test[feat] = df_test[feat].astype("category") #le.fit_transform(df_train[feat].astype(str))
    
df_test.drop(columns = df_test.select_dtypes(include = "float").loc[:, df_test.select_dtypes(include = "float").std() < .1], inplace=True)

In [46]:
probas_ = cl.predict_proba(df_test.iloc[:,:-1])

In [60]:
sub = pd.read_csv("sample_submission.csv", index_col=0)

sub["Predicted"] = lh.inverse_transform(np.argmax(probas_, axis=1))

In [62]:
sub.to_csv("vanilla_lgbm_preds.csv")

In [37]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder 

ohe = LabelEncoder()
y_lh = ohe.fit_transform(y)
y_test_lh = y_lh[test]

accuracy_score(y_test_lh,np.argmax(probas_, axis=1))
#confusion_matrix(y_test_lh,np.argmax(probas_, axis=1))